In [1]:
from db import duck_query_df
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tableone import TableOne

all_tabs = duck_query_df("SHOW ALL TABLES;")
all_tabs.head()


,database,schema,name,column_names,column_types,temporary
0,db,mimiciv_derived,acei,"[subject_id, hadm_id, acei, starttime, stoptime]","[INTEGER, INTEGER, VARCHAR, TIMESTAMP, TIMESTAMP]",False
1,db,mimiciv_derived,age,"[subject_id, hadm_id, admittime, anchor_age, a...","[INTEGER, INTEGER, TIMESTAMP, SMALLINT, SMALLI...",False
2,db,mimiciv_derived,antibiotic,"[subject_id, hadm_id, stay_id, antibiotic, rou...","[INTEGER, INTEGER, INTEGER, VARCHAR, VARCHAR, ...",False
3,db,mimiciv_derived,bg,"[subject_id, hadm_id, charttime, specimen, so2...","[INTEGER, INTEGER, TIMESTAMP, VARCHAR, DOUBLE,...",False
4,db,mimiciv_derived,blood_differential,"[subject_id, hadm_id, charttime, specimen_id, ...","[INTEGER, INTEGER, TIMESTAMP, INTEGER, DOUBLE,...",False


In [ ]:
hosp = duck_query_df("""
SELECT
    pat.subject_id,
    adm.hadm_id,
    DENSE_RANK() OVER (PARTITION BY pat.subject_id ORDER BY adm.admittime) AS hosp_stay_num,
    CASE
        WHEN FIRST_VALUE(adm.hadm_id) OVER (PARTITION BY pat.subject_id ORDER BY adm.admittime) = adm.hadm_id THEN 1
        ELSE 0
    END AS pat_count,
    pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age,
    pat.gender,
    EXTRACT(EPOCH FROM (adm.dischtime - adm.admittime)) / 3600 / 24 AS hosp_los,
    pat.dod,
    (pat.dod::date - adm.dischtime::date) AS days_to_death,
    -- mortality flags
    CASE WHEN (pat.dod::date - adm.dischtime::date) = 0 THEN 1 ELSE 0 END AS hospital_mortality
FROM db.mimiciv_hosp.patients pat
INNER JOIN db.mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id;
""")
hosp.head()

,subject_id,hadm_id,hosp_stay_num,pat_count,age,gender,insurance,hosp_los,dod,days_to_death,hospital_mortality
0,12469804,25529826,24,0,37,M,Private,4.688194,NaT,<NA>,0
1,12469804,27886494,25,0,37,M,Private,2.286806,NaT,<NA>,0
2,12469804,24837653,26,0,37,M,Private,2.661111,NaT,<NA>,0
3,12469804,26228006,27,0,40,M,Medicaid,3.039583,NaT,<NA>,0
4,12469804,22290520,28,0,40,M,Medicaid,1.822917,NaT,<NA>,0


In [ ]:
# Add column for 1 year mortality
hosp['one_year_mortality'] = hosp['days_to_death'].notnull().astype(int)

# Temp dataframe with the days to death for only the last ICU stay
last_dod = hosp.groupby('subject_id')[['hosp_stay_num']].max().reset_index()
last_dod = last_dod.merge(hosp[['subject_id', 'hosp_stay_num', 'days_to_death']],
                          on=['subject_id', 'hosp_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

hosp = hosp.merge(last_dod, how='left', on=['subject_id', 'hosp_stay_num'])
del last_dod
hosp.sort_values(['subject_id', 'hosp_stay_num'], inplace=True)

# fix some data type issues
int_cols = hosp.dtypes.values=="Int64"
hosp.loc[:, int_cols] = hosp.loc[:, int_cols].astype(float)
hosp.loc[:, int_cols] = hosp.loc[:, int_cols].astype(int, errors="ignore")

hosp.head()

        subject_id   hadm_id  hosp_stay_num  pat_count  age gender insurance  \
433351    10000032  22595853              1          1   52      F  Medicaid   
433352    10000032  22841357              2          0   52      F  Medicaid   
433353    10000032  29079034              3          0   52      F  Medicaid   
433354    10000032  25742920              4          0   52      F  Medicaid   
503835    10000068  25022803              1          1   19      F      None   

        hosp_los        dod  days_to_death  hospital_mortality  \
433351  0.786111 2180-09-09            125                   0   
433352  1.015278 2180-09-09             74                   0   
433353  2.222222 2180-09-09             46                   0   
433354  1.754167 2180-09-09             33                   0   
503835  0.298611        NaT           <NA>                   0   

        one_year_mortality  days_to_death_last_stay_id  
433351                   1                        <NA>  
433352  

In [ ]:
icu = duck_query_df("""
SELECT
    pat.subject_id,
    adm.hadm_id,
    icu.stay_id,
    ROW_NUMBER() OVER (PARTITION BY pat.subject_id ORDER BY icu.intime) AS icu_stay_num,
    DENSE_RANK() OVER (PARTITION BY pat.subject_id ORDER BY adm.admittime) AS hosp_stay_num,
    CASE
        WHEN FIRST_VALUE(icu.stay_id) OVER (PARTITION BY pat.subject_id ORDER BY icu.intime) = icu.stay_id THEN 1
        ELSE 0
    END AS pat_count,
    pat.anchor_age + (EXTRACT(YEAR FROM icu.intime) - pat.anchor_year) AS age,
    pat.gender,
    icu.first_careunit,
    icu.los AS icu_los,
    EXTRACT(EPOCH FROM (adm.dischtime - adm.admittime)) / 3600 / 24 AS hosp_los,
    pat.dod,
    (pat.dod::date - adm.dischtime::date) AS days_to_death,
    -- mortality flags
    CASE WHEN (pat.dod::date - adm.dischtime::date) = 0 THEN 1 ELSE 0 END AS hospital_mortality,
    CASE WHEN (pat.dod::date - icu.outtime::date) = 0 THEN 1 ELSE 0 END AS icu_mortality
FROM db.mimiciv_hosp.patients pat
INNER JOIN db.mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id
INNER JOIN db.mimiciv_icu.icustays icu
    ON adm.hadm_id = icu.hadm_id;
""")
icu.head()

,subject_id,hadm_id,stay_id,icu_stay_num,hosp_stay_num,pat_count,age,gender,insurance,first_careunit,icu_los,hosp_los,dod,days_to_death,hospital_mortality,icu_mortality
0,13425009,22460020,34340499,1,1,1,78,F,Medicare,Medical/Surgical Intensive Care Unit (MICU/SICU),1.087859,6.849306,2134-10-30,190,0,0
1,13425009,22691309,30232276,2,2,0,78,F,Medicare,Medical/Surgical Intensive Care Unit (MICU/SICU),1.318484,24.720833,2134-10-30,141,0,0
2,13426633,22747154,34027287,1,1,1,38,M,Private,Trauma SICU (TSICU),2.567905,3.697222,NaT,<NA>,0,0
3,13429103,28102320,33870713,1,1,1,91,F,Medicare,Trauma SICU (TSICU),2.273472,6.681944,2178-04-05,88,0,0
4,13430836,26055880,36199501,1,1,1,23,M,None,Medical Intensive Care Unit (MICU),0.409502,0.481250,2180-12-22,0,1,1


In [ ]:
# add 1 year mortality
icu['one_year_mortality'] = icu['days_to_death'].notnull().astype(int)

# create a dataframe with the days to death for only the last ICU stay
last_dod = icu.groupby('subject_id')[['icu_stay_num']].max().reset_index()
last_dod = last_dod.merge(icu[['subject_id', 'icu_stay_num', 'days_to_death']], on=['subject_id', 'icu_stay_num'], how='inner')
last_dod.rename(columns={'days_to_death': 'days_to_death_last_stay_id'}, inplace=True)

icu = icu.merge(last_dod, how='left', on=['subject_id', 'icu_stay_num'])
del last_dod
icu.sort_values(['subject_id', 'icu_stay_num'], inplace=True)

# add a grouping variable for table one so we can have hospital mortality as a group and a row
icu['hosp_mort'] = icu['hospital_mortality']

# fix some data type issues
int_cols = icu.dtypes.values=="Int64"
icu.loc[:, int_cols] = icu.loc[:, int_cols].astype(float)
icu.loc[:, int_cols] = icu.loc[:, int_cols].astype(int, errors="ignore")

icu.head()

In [ ]:
columns = [
    "pat_count",
    # , "hadm_count",
    "age", "gender",
    # "first_careunit",
    # "icu_los",
    "hosp_los",
    # 'icu_mortality',
    'hospital_mortality',
    "one_year_mortality",
    # "days_to_death_last_stay_id"
]

categorical = [
    "pat_count",
    # "hadm_count",
    "gender",
    # "first_careunit",
    # mortality flags
    # 'icu_mortality',
    'hospital_mortality',
    'one_year_mortality',
]

order = {
    "pat_count": ['1', '0'],
    # "hadm_count": [1, 0],
    "gender": ["F", "M"],
    # "icu_mortality": [1, 0],
    "hospital_mortality": ['1', '0'],
    "one_year_mortality": ['1', '0'],
}

limit = {
    "pat_count": 1, # "hadm_count": 1,
    # "gender": 1,
    # "icu_mortality": 1,
    "hospital_mortality": 1,
    "one_year_mortality": 1,
}

rename = {
    "pat_count": "Distinct patients", "hadm_count": "Distinct hospitalizations",
    "age": "Age", "gender": "Administrative Gender", "insurance": "Insurance",
    "first_careunit": "First ICU stay, unit type",
    "icu_los": "ICU length of stay", "hosp_los": "Hospital length of stay",
    "icu_mortality": "In-ICU mortality",
    "hospital_mortality": "In-hospital mortality",
    "one_year_mortality": "One year mortality",
    # "days_to_death_last_stay_id": "Time to death (days)",
}

print('ICU demographics')
icu_table = TableOne(icu, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename)
display(icu_table)
print('Hospital demographics')
hosp_table = TableOne(hosp, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename)
display(hosp_table)

compare_table = hosp_table.tableone.drop(columns='Missing').copy()
compare_table.rename(columns={'Overall': 'Hospital stays'}, inplace=True)

compare_table = compare_table.merge(
    icu_table.tableone.drop(columns='Missing'),
    left_index=True, right_index=True, how='inner'
)
compare_table.rename(columns={'Overall': 'ICU stays'}, inplace=True)
display(compare_table)

ICU demographics


Missing       Overall
n                                                    94458
Distinct patients, n (%)           1          65366 (69.2)
Age, mean (SD)                             0   64.8 (16.7)
Administrative Gender, n (%)       F          41583 (44.0)
                                   M          52875 (56.0)
Hospital length of stay, mean (SD)         0   11.8 (15.5)
In-hospital mortality, n (%)       1          11616 (12.3)
One year mortality, n (%)          1          37967 (40.2)

Hospital demographics


Missing        Overall
n                                                    546028
Distinct patients, n (%)           1          223452 (40.9)
Age, mean (SD)                             0    59.2 (19.1)
Administrative Gender, n (%)       F          284097 (52.0)
                                   M          261931 (48.0)
Hospital length of stay, mean (SD)         0      4.8 (7.2)
In-hospital mortality, n (%)       1            12149 (2.2)
One year mortality, n (%)          1          144966 (26.5)

In [2]:
print("Charlson Comorbidity Index Table:")
charlson = duck_query_df("SELECT * FROM db.mimiciv_derived.charlson;")
charlson.head()

print("First Day Labs Table:")
first_labs = duck_query_df("SELECT * FROM db.mimiciv_derived.first_day_lab;")
first_labs.head()

Charlson Comorbidity Index Table:
First Day Labs Table:


,subject_id,stay_id,hematocrit_min,hematocrit_max,hemoglobin_min,hemoglobin_max,platelets_min,platelets_max,wbc_min,wbc_max,...,bilirubin_indirect_min,bilirubin_indirect_max,ck_cpk_min,ck_cpk_max,ck_mb_min,ck_mb_max,ggt_min,ggt_max,ld_ldh_min,ld_ldh_max
0,12466550,30000153,29.1,39.1,9.8,13.0,162.0,177.0,15.2,17.9,...,NaN,NaN,605.0,605.0,20.0,20.0,NaN,NaN,NaN,NaN
1,13180007,30000213,23.9,27.6,7.4,8.5,219.0,243.0,5.7,7.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,300.0
2,18421337,30000484,24.6,31.0,8.1,9.9,357.0,403.0,24.2,30.1,...,NaN,NaN,40.0,74.0,NaN,NaN,NaN,NaN,419.0,419.0
3,12207593,30000646,37.8,39.3,12.9,13.2,266.0,337.0,7.9,10.6,...,NaN,NaN,64.0,64.0,1.0,1.0,NaN,NaN,NaN,NaN
4,15726459,30000831,38.0,40.8,12.4,13.3,285.0,311.0,14.2,21.3,...,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN
